## <span style="color:red">*Important! Do this before proceeding.* </span>

Before starting the tutorial, please make a backup copy of this notebook and put it in the "Summer School Materials" folder that should be in your home directory. Save it with the same name, but add your initials at the end of the filename. You can do this by dragging the notebook from the shared drive into your "Summer School Materials" folder using File Explorer.

# <span style="color:blue">Welcome to Tutorial #3: Diffraction Simulation</span>

Electron diffraction patterns provide valuable information about the symmetry and structure of materials. Here we will show you the basic tools for simulating diffraction patterns and atomic-resolution images using the multislice method, through the open-source software [MuSTEM](https://github.com/HamishGBrown/MuSTEM). MuSTEM can be run as a precompiled executable directly in Windows, or through Wine in Mac and (probably) Linux. By the end of this tutorial, we hope you have a better intuition for diffraction simulation, and will be able to use this tool in your own research. 

In Sections 1-3, you will learn about diffraction simulation through running pre-assembled code. In Section 4, you will get to run a simulation from scratch, defining the input crystal structure yourself. If you don't get to Section 4, don't worry about it! Sections 1-3 will still give you a good introduction. 

Throughout the tutorial there will be checkpoints asking you to reflect on what you see or think about some questions. Feel free to discuss these questions with the others in the breakout room or a TA!

## Table of contents: <a name="toc"/>
**[Section 1: Using CBED simulations to detect broken inversion symmetry in 2D materials](#section1)**  
* [1.1. Setting up and running a basic CBED simulation](#section1.1)
* [1.2. Loading and viewing the simulated CBED patterns](#section1.2)
* [1.3. Detecting broken inversion symmetry](#section1.3)

**[Section 2: Varying the simulation parameters to tune your experiment](#section2)**  
* [2.1. Changing the aperture size](#section2.1)
* [2.2. Changing the supercell size](#section2.2)
* [2.3. Changing the crystal thickness](#section2.3) 
* [2.4. Changing the crystal tilt](#section2.4) 
 * [2.4.1 Tilt effects on a thick sample](#section2.4.1)
 * [2.4.2 Tilt effects on a thin sample](#section2.4.2)

**[Section 3: Simulating atomic-resolution STEM images](#section3)**  
* [3.1. How to simulate STEM images](#section3.1) 
* [Recap of what we've learned](#sectionRecap) 

**[Section 4: Starting from scratch](#section4)**
* [4.1. Making your own .xtl and driver file](#section4.1)
* [4.2. Define a CBED simulation](#section4.2)
* [4.3. Define a STEM image simulation](#section4.3)
* [4.4. Hexagonal unit cell](#section4.4)
* [4.5. Running MuSTEM from the command line](#section4.5)

*Note on notation: Inside the code cells, places where you should change the variables are marked by `#--- User input ---#`.*

*Note on code: To make loading the simulation results less tedious, we've written a method that reads in the diffraction data for you. To avoid confusing the method, it's best to change the output name of the simulated file each time you re-run the code with a different parameter.*

### Set paths and import relevant packages

We will be using the methods defined in the "MuSTEM_methods.py" file. For the purposes of this tutorial, you can run this cell without changing anything. 

If you want to use this notebook on your own version of MuSTEM, you will need to alter the variable ***`mustem_path`*** so it points to the folder path where your own MuSTEM executable is located. 

We are setting the ***`device_type`*** variable here also. MuSTEM has a standard CPU version, and an accelerated GPU version. We will use the CPU version for this summer school. If you want to download MuSTEM at home and have an Nvidia GPU that meets the necessary specs, you can set ***`device_type = 1`*** to indicate that you are using the GPU version. 

We are also defining the global ***`folder path`*** under which everything will be saved. You can edit this later on your own file systems.


In [ ]:
#Setup: import relevant packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import subprocess
import shlex
import os
import glob
import matplotlib.patches as patches
from scipy import ndimage
from MuSTEM_methods import *  #<-- All the needed methods 
%matplotlib inline

#Location of mustem : change as needed
#On Macs: navigate to the MuSTEM .exe directory, then type "winepath ./" into Terminal. Copy result here. 

device_type = 0 #GPU: 1, CPU: 0
mustem_path = "C:/Users/paradim2/CPU_muSTEM_x64_v5.3_single_precision/CPU_muSTEM_x64_v5.3_single_precision/CPU_muSTEM_v5.3_single_precision.exe"
folder_path = "../Data/"

# <span style="color:purple">Section 1: Detecting broken inversion symmetry in 2D materials</span> <a name="section1"/>

To build some intuition and excitement for CBED simulation, we will work through an example involving two popular 2D materials: MoS<sub>2</sub>, and graphene. 

The crystal structure of monolayer MoS<sub>2</sub> lacks inversion symmetry: if you rotate the structure by 180°, there is no way to translate the atoms such that the original structure is recovered. 

This broken symmetry manifests as an intensity asymmetry between Friedel pairs in the electron diffraction pattern. In other words, some of the diffraction peaks directly opposite each other will have different intensities even when the crystal is perfectly on zone axis! **Electron diffraction is a powerful tool for detecting and quantifying polarity in 2D materials.**

<div>  
<img src="Figures/MoS2_brokensymmetry4.png" width="500"/>
</div>

While monolayer MoS<sub>2</sub> and some other transition metal dichalcogenides have broken inversion symmetry, some variants of multilayer MoS<sub>2</sub>, such as the 2H stacking, are centrosymmetric. Similarly, other 2D materials such as graphene are centrosymmetric. Being able to quickly differentiate between a centrosymmetric and a non-centrosymmetric structure in diffraction space can be essential in differentiating between different materials, phases, or stacking orders in your sample. 

<div>  
<img src="Figures/MoS2_Vs_Graphene.png" width="500"/>
</div>

In this section you will 
* *1)* Run multislice simulations of monolayer MoS<sub>2</sub> and graphene, and learn how to manipulate and understand the diffraction data.
* *2)* Learn how to detect broken inversion symmetry in the diffraction patterns.

*To learn more about detecting polarity in diffraction space, check out this paper: [https://doi.org/10.1016/j.ultramic.2020.113019](https://doi.org/10.1016/j.ultramic.2020.113019)*

[Go to top](#toc)

## <span style="color:purple">1.1. Setting up a diffraction simulation in MuSTEM</span> <a name="section1.1"/> 

MuSTEM needs three input files: **1) a "user input" file, 2) an .xtl file, and 3) a "driver" file.**

*1)* The **"user input"** file is a program control file, and must always be called ***`user_input.txt`***. It directs whether MuSTEM runs interactively or in a pre-defined manner using an existing driver file. To create `user_input.txt`, make a new text file and enter one of these words/phrases in the first line: 

> 1. ***`Interactive`***: You prompt the simulation to run and fill out the parameters in real time from the command line. 
> 2. ***`Record`***: Same as `Interactive`, but records the simulation driver file based on your inputs. However it will not overwrite an existing driver file.
> 3. ***`Record overwrite`***: Same as `Record`, but will overwrite an existing simulation driver file. 
> 4. ***`Play`***: If you chose this option, you would put the name of a driver file in the line below it. The program would then run the simulation on that driver file, with no user input needed. 
> 5. ***`Play all`***: If you chose this option, you would put the name of multiple driver files in the lines below it (one per line). The program would then run the simulation on all the listed driver files. For example, it could look like this:
>>Play all<br>
   Driver1.txt<br>
   Driver2.txt
   
Here we have already created the program control file for you, and you can see it by running the cell below. 

*2)* The **.xtl** file contains a list of the atomic positions (in fractional coordinates) needed to define a rectangular unit cell for the crystal. We have provided a .xtl file for monolayer MoS<sub>2</sub> under "Xtl_files/MoS2_mono.xtl". This will be explained in more detail after the following cell.

<span style="color:purple">Run the cells below to view the xtl and user input files for monolayer MoS<sub>2</sub>.</span>

[Go to top](#toc)

In [ ]:
#Open .xtl file
print("XTL FILE:\n")
xtl_file = open(folder_path + "Xtl_files/MoS2_mono.xtl")
print(xtl_file.read())
xtl_file.close()

#Open user input file
print("USER INPUT FILE:\n")
ui_file = open(folder_path + "user_input.txt")
print(ui_file.read())
ui_file.close()

### <span style="color:purple">1.1 (continued)</span>
Each non-blank line in the **.xtl** file above conveys information about the specimen. In order:

> 1. ***`MoS2_mono_for_muSTEM super cell`***: Name of file + other comments: for organization
> 2. ***`3.1700 5.4906 3.063 90.0 90.0 90.0`***: a, b, c, $\alpha$, $\beta$, $\gamma$. Unit cell dimensions in Angstroms and angles in degrees, where the angles *must* be 90° (rectangular unit cell).
> 3. ***`2`***: Number of distinct elements (ex: 2 for MoS<sub>2</sub>)
> 4. ***`Mo`***: Symbol for element 1
> 5. ***`2   42    1.000   0.0063326`***: Number atoms of element 1, atomic number, occupancy, RMS atomic displacements in Angstroms<sup>2</sup> due to temperature.
> 6. ***`0.00000 0.00000 0.50000 ...`***: Fractional coordinates of each atom of element 1 (all values between 0-1). 
> 7. Repeat for remaining element types in specimen.

The spaces, blank lines, and capitalization are all important! Also note that the input crystal structure must have a *rectangular* unit cell for multislice diffraction, *no matter what the symmetry of the crystal is*. If your structure has a different unit cell shape, like hexagonal or rhombahedral, you must transform the cell to be rectangular.

*3)* The **driver file** tells MuSTEM what microscope and sample parameters to use for the simulation, ex: sample thickness, beam voltage, beam tilt, detector size, etc. 

The section below allows you to create a driver file for CBED simulation. We have already set up the correct parameters for simulating a CBED pattern from monolayer MoS2 at 80 keV beam energy. Note that we have hard-coded in much of the driver file, leaving only the essential parameters to be varied. You will have a chance to edit the full parameter space at the end of this tutorial. 

<span style="color:purple">Run the cell below to make and view the driver file.</span>

[Go to top](#toc)

In [ ]:
#...Make a driver file for CBED in MuSTEM...# 

#.Set input and output filenames and locations
fnm_prefix = "01_MoS2_mono" #<-- Filename prefix for the simulation
driver_name = folder_path + "Drivers/"+fnm_prefix+"_driver.txt" #<-- Name of the driver file
outfnm = folder_path + "Results/1p1_MoS2/"+fnm_prefix #<-- Name of simulated output
infnm = folder_path + "Xtl_files/MoS2_mono.xtl" #<-- Name of input .xtl file 

#.Change microscope and sample parameters
kV = "80" #accelerating voltage
numSlice = "10" #number of slices per unit cell
thickness = "3.06300" #Thickness in Angstrom. For series: "Thickness start:Thickness end:Step"
tile = ["26", "15"] #Tile supercell in [X, Y]
numPix = ["1024", "1024"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "3"  #aperture semiangle in mrad
doTilt = [["1","0","0"], ["2","0","0"]] #Tilt: ["type", "polar", "azimuthal "]. Type "1": beam, "2": specimen (mrad)

#.Assemble the parameters into one list
params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

#.Make the driver file using the chosen parameters above
make_driver_cbed(driver_name, params, device_type)

#.Print the driver file you just created
print("DRIVER FILE:\n")
driver_file = open(driver_name)
print(driver_file.read())
driver_file.close()

###  <span style="color:purple">1.1. (continued): The driver file</span>

As you can see above, the driver file has a lot of different options for running the simulation. We've hard-coded the parameters that should be reliably safe to use most of the time. However, you have the ability to change these variables in the code:

> 1. ***`outfnm`***: The filename prefix of the output simulation. Here this value was: `Results/1p1_MoS2/01_MoS2_mono`
> 2. ***`infnm`***: The filename of the input crystal structure file. Here this value was: `Xtl_files/MoS2_mono.xtl`
> 3. ***`kV`***: The electron beam voltage in kV. Here this value was: `80`
> 4. ***`numSlice`***: The number of slices per unit cell. The heavier and more complicated the unit cell, the more slices you need. Using at least 4-5 slices per unit cell is ideal. Too few, and the weak-phase approximation that is central to multlislice will fail. Here this value was: `10`
> 5. ***`thickness`***: The thickness of the crystal you're simulating in Angstroms. Here this value was: `3.063` (one unit cell thickness)
> 6. ***`tile`***: The number of times the simulation tiles your unit cell to form a bigger sample. (More on this later!) Here this value was: `[26, 15]` 
> 7. ***`numPix`***: The number of diffraction-space pixels (sort of like your detector size). This improves your diffraction-space resolution. It's best to use at least `512x512` pixels for CBED simulation, and at least `1024x1024` for a reliable HAADF simulation. In this tutorial, we may occasionally use smaller mesh sizes for the purpose of increasing the simulation speed. Here this value was: `1024x1024`
> 8. ***`aperture`***: Your beam convergence angle in milliradians. (More on this later!) Here this value was: `3`
> 9. ***`doTilt`***: Tilt the specimen and/or the beam to get to different zone axes. (More on this later!) Here this value was: `0` (no tilt)

We will explore the effects of changing some of these values throughout this tutorial.

### <span style="color:purple">Running MuSTEM </span>

Now that we've created the driver file above, we can finally run a simulation. Normally, MuSTEM is run from the command line. Here, we've piped the command line output to this notebook, so you can see what the software is doing. 

<span style="color:purple">Run the cell below in ***`verbose=True`*** mode to see the normal MuSTEM output. Scroll through the text to see where each of the driver file parameters defined above came into use.</span>

(Later on in the tutorial, if you ever want to run a cell without starting the MuSTEM software, comment out the line with the ***`run_mustem`*** method. It takes about 2-3 minutes for each simulation to run.)

[Go to top](#toc)

In [ ]:
#...Run MuSTEM without pauses using the driver file above...#

verbose = True  #If True, print entire MuSTEM Terminal output. Else, print condensed version (faster)
run_mustem(mustem_path + " nopause", verbose) #WINDOWS

### <span style="color:purple">Checkpoint 1</span>

* In the simulation output, could you find where each of the parameters in the driver file was used? 

* At the end of the simulation you should see a line containing `Intensity: 0.996`. This is the intensity of the final exit wave in multislice with respect to the incident wave. What does it mean if this number is less than 1? 

* As you may have noticed above, we are including the effects of `absorption`, i.e. attenuating the wave function to account for electrons lost to thermal scattering. How would you expect the `Intensity` value to change if we hadn't included these effects? 

 Though we haven't shown it here, it's often a good idea to run your simulation without including absorption to check if the `Intensity` is equal to 1. If it isn't, there's something up with your simulation parameters (ex: your diffraction grid size is too small).

## <span style="color:purple">1.2. Viewing the MoS<sub>2</sub> diffraction pattern</span> <a name="section1.2"/> 

When the simulation is finished, MuSTEM outputs a 32-bit real binary file (".bin" extension), which can support an intensity range of 1:2<sup>32</sup>. This allows us to see both the bright, central disk, and the weaker scattered disks at the same time. The bytes are arranged in big-endian byte order (don't worry about this here). 

Below, the ***`load_files`*** method reads in the binary CBED simulation file you just made, using the output filename defined in the previous section as a reference. The ***`log_cbed`*** method takes the log of the CBED pattern, to allow the high-angle diffraction spots and the central spot to be viewed at the same time. The ***`crop_cbed`*** method then crops the CBED pattern so that we only see the first two rings. 

<span style="color:purple">Run the cell below to load and view the simulation file. Feel free to change the cropped size of the diffraction pattern, and the maximum and minimum displayed intensity values as well. These are within the dashed lines labeled `--- User inputs ---`.</span>

*Note: every time you re-run a simulation with different driver file parameters, either delete the old simulation file or **use a new output filename**, to not confuse the `load_files` method.*



[Go to top](#toc)

In [ ]:
#...Plot the simulated MoS2 diffraction pattern on linear and log scales...# 

#------------------------ User inputs ----------------------------#

crop_size = 150  #<-- Cropped size of diffraction pattern (DP)

vmax_lin = 2e-5 #<-- Max intensity shown in linear DP
vmin_lin = 1e-7 #<-- Min intensity shown in linear DP

vmax_log = -11 #<-- Max intensity shown in log DP
vmin_log = -14 #<-- Min intensity shown in log DP

#------------------------- End user input -------------------------#

#.Read in all cbed files in the produced folder
fnm_prefix = "01_MoS2_mono"
outfnm = folder_path + "Results/1p1_MoS2/"+fnm_prefix #<-- Name of simulated output
cbed_mos2 = load_files(outfnm) #<-- Load files using the prefix you set in the driver file
cbed_mos2_log  = log_cbed(cbed_mos2)  #<-- Take log of cbed pattern

#.Set up the figure:
fig, ax = plt.subplots(1,2, figsize=(15, 7)) 

#.Plot linear and log scale images
im1 = ax[0].imshow(crop_cbed(cbed_mos2,crop_size), vmin=vmin_lin, vmax=vmax_lin, cmap='CMRmap') 
ax[0].set_title("MoS2 CBED pattern: Linear scale")
im2 = ax[1].imshow(crop_cbed(cbed_mos2_log, crop_size), vmin=vmin_log, vmax=vmax_log, cmap='CMRmap')
ax[1].set_title("MoS2 CBED pattern: Log scale")
cb1 = fig.colorbar(im1, ax=ax[0])
cb2 = fig.colorbar(im2, ax=ax[1])

### <span style="color:purple">Checkpoint 2</span>

* Does the symmetry of the MoS<sub>2</sub> diffraction pattern match what you expect (or have experienced)? 

* Going back to the original question, does the monolayer MoS<sub>2</sub> diffraction pattern have inversion symmetry? If not, which Friedel pairs have asymmetric contrast? Which pairs are symmetric?

## <span style="color:purple">Detecting broken inversion symmetry in MoS<sub>2</sub></span>

Can we more clearly detect the broken inversion symmetry in monolayer MoS<sub>2</sub>? Our method for checking if a diffraction pattern is centrosymmetric or not is to rotate diffraction pattern by 180°, and then subtract it from the original diffraction pattern *(in linear, not log, scale)*. If the crystal is centrosymmetric, the peaks in each Friedel pair have *equal* intensity. If the inversion symmetry is broken, then the peaks will have *unequal* intensity.

<span style="color:purple">Run the cell below to subtract the simulated CBED pattern from its 180°-rotated version. You should see some peaks appearing in the difference image, representing Friedel pairs of unequal intensity. The magnitude of the asymmetry is related to the difference in atomic number between Mo and S.</span> 

[Go to top](#toc)

In [ ]:
#.Rotate and subtract the diffraction pattern 
cbed_m_r180 = np.roll(np.rot90(cbed_mos2,2),(1,1), axis=(0,1)) #<--Rotate diffraction pattern by 180 degrees
cbed_m_mr180 = np.subtract(cbed_mos2, cbed_m_r180)  #<--Subtract the original and rotated patterns

#.Plot the subtracted diffraction pattern
crop_size = 256
plt.imshow(crop_cbed(cbed_m_mr180, crop_size), cmap='RdBu_r', vmin=-1e-6, vmax=1e-6)
plt.title("MoS2 CBED Difference Image")
plt.colorbar()

### <span style="color:purple">Checkpoint 3</span>

* You should see that there are many peaks in the MoS<sub>2</sub> CBED difference image, each one representing a broken inversion symmetry in the original lattice. Looking back at the original MoS<sub>2</sub> CBED simulations, are there any reflections in the difference image that have zero intensity? Which ones are they?

<div>  
<img src="Figures/MoS2_DiffImg_Pratiti_3.jpg" width="500"/>
</div>

* The figure above shows an experimental CBED pattern from MoS<sub>2</sub>, which shows the characteristic breaking of Friedel's law that you just simulated. As explained in the referenced paper, we can think of the higher-order diffraction peaks as the result of different multiple scattering paths in reciprocal space. While the first ring of MoS<sub>2</sub> peaks have an asymmetry, the second ring of peaks is symmetric in intensity because their multiple scattering paths are identical. These symmetric peaks are the ones that have zero intensity in the difference image above. 

## <span style="color:purple">1.3: What about a centrosymmetric structure like graphene?</span> <a name="section1.3"/> 

We were able to detect a breaking of inversion symmetry in the monolayer MoS<sub>2</sub>. What happens if we try a centrosymmetric structure instead? Unlike monolayer MoS<sub>2</sub>, monolayer graphene does have inversion symmetry. Let's compare CBED patterns from monolayer MoS<sub>2</sub> vs. graphene. 

<span style="color:purple">Run the cell below to see a CBED simulation of monolayer graphene. Look over the driver file parameters: which are the same and which are different from the MoS<sub>2</sub> simulation?</span>

[Go to top](#toc)

In [ ]:
#... Make the graphene driver file and run the simulation ...#

#.Set input and output filenames and locations
fnm_prefix = "01_Graphene_mono"
driver_name = folder_path + "Drivers/"+fnm_prefix+"_driver.txt" #<-- Name of the driver file
outfnm = folder_path + "Results/1p1_Graphene/"+fnm_prefix #<-- Name of simulated output
infnm = folder_path + "Xtl_files/Graphene_mono.xtl" #<-- Name of input .xtl file (don't change)

#.Change microscope and sample parameters
kV = "80" #accelerating voltage
numSlice = "5" #number of slices per unit cell
thickness = "8.685" #Thickness = 1 unit cell.
tile = ["33", "19"] #Tile supercell in [X, Y]
numPix = ["1024", "1024"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "3"  #aperture semiangle in mrad
doTilt = [["1","0","0"],["2","0","0"]] #tilt: [["1"=beam , polar, azimuthal], ["2"=specimen, pol., azimuth.]] (mrad)

#.Assemble the parameters into one list
params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

#.Make the driver file using the chosen parameters above
make_driver_cbed(driver_name, params, device_type)

#.Run MuSTEM
run_mustem(mustem_path + " nopause", True)

Now run the cell below to load the simulation you just ran.

In [ ]:
#.Load and plot the simulation file
cbed_graphene = load_files(outfnm) #<-- Load file using the filename prefix from above
crop_size_gr = 150 #<-- Crop Graphene pattern
crop_size_Mo = 128 #<-- Crop MoS2 pattern

#. Plot MoS2 CBED
fig, ax = plt.subplots(1,2, figsize=(15, 7)) 
im1 = ax[0].imshow(log_cbed(crop_cbed(cbed_mos2,crop_size_Mo)), cmap='CMRmap', vmin=-14, vmax=-11) 
ax[0].set_title("MoS2: CBED pattern (LOG)")
cb1 = fig.colorbar(im1, ax=ax[0])

#. Plot graphene CBED
im2 = ax[1].imshow(log_cbed(crop_cbed(cbed_graphene, crop_size_gr)), cmap='CMRmap', vmin=-16, vmax=-11)
ax[1].set_title("Graphene: CBED pattern (LOG)")
cb2 = fig.colorbar(im2, ax=ax[1])

### <span style="color:purple">1.3. (continued): Comparing graphene and MoS<sub>2</sub> </span>

As we did for MoS<sub>2</sub>, we'll rotate the graphene diffraction pattern by 180° and subtract it from the original. A structure with broken inversion symmetry would show contrast in the subtracted pattern, but a centrosymmetric structure should exactly cancel out when it's subtracted. 

<span style="color:purple">Run the cell below to compare the subtracted CBED patterns of MoS<sub>2</sub> and graphene.</span>

*Note: If the intensity of the lowest-order peaks in the difference image is less than 0.1% of the original diffraction peaks, the effect may be due to numerical error in the simulation. In that case, also make sure the result is robust when you vary the diffraction mesh size, number of slices, and unit cell tiling!*

[Go to top](#toc)

In [ ]:
#.Rotate and subtract the diffraction pattern 
cbed_g_r180 = np.roll(np.rot90(cbed_graphene,2),(1,1), axis=(0,1)) #<--Rotate diffraction pattern by 180 degrees
cbed_g_mr180 = np.subtract(cbed_graphene, cbed_g_r180)  #<--Subtract the original and rotated patterns

#.Plot the subtracted diffraction pattern
crop_size = 256

fig, ax = plt.subplots(1,2, figsize=(15, 7)) 
im1 = ax[0].imshow(crop_cbed(cbed_m_mr180,crop_size), cmap='RdBu_r', vmin=-1e-6, vmax=1e-6) 
ax[0].set_title("MoS2: subtracted CBED pattern")

im2 = ax[1].imshow(crop_cbed(cbed_g_mr180, crop_size), cmap='RdBu_r', vmin=-1e-7, vmax=1e-7)
ax[1].set_title("Graphene: subtracted CBED pattern")

cb1 = fig.colorbar(im1, ax=ax[0])
cb2 = fig.colorbar(im2, ax=ax[1])

### <span style="color:purple">Checkpoint 4</span>

* Do you see any contrast in the monolayer graphene difference image? 

Of course, our experimental data will never be this clean. In reality, there are different experimental effects that could distort the data and produce a non-zero difference image even for centrosymmetric structures. In the next section, we will learn how to change the parameters of the multislice simulation to more accurately emulate the irregularities of a real experiment. 

# <span style="color:orange"> Section 2: Changing simulation parameters </span><a name="section2"/> 

Now that we've hopefully got you intruigued by the power of diffraction simulation, it's time to start playing with the parameters! You can tune the simulation to carefully match your microscope settings and sample characteristics. In this section, we will walk you through varying the convergence angle, sample size, sample thickness, and sample tilt. 

There are some parameters that we won't change: the diffraction-space mesh size (***`numPix`***) and the number of slices per unit cell (***`numSlice`***). For a trustworthy simulation, it's ideal to use a mesh size of `1024x1024` or `2048x2048` pixels and at least `4-5` slices per unit cell. Heavier atoms require more slices to be accurately represented with the weak phase approximation. If you are going to be tilting your sample by a lot, it's also a good idea to increase the number of slices per unit cell. 

[Go to top](#toc)

## <span style="color:orange">2.1. Changing the aperture size </span><a name="section2.1"/>

Two important values to consider in a diffraction experiment are the *aperture size* and the *beam convergence angle*, which affect your real-space and diffraction-space resolution. A larger convergence angle leads to larger CBED disks and a smaller real-space probe, while a smaller convergence angle creates smaller CBED disks. On some microscopes, it's possible to vary the aperture size and convergence angle independently, while on others, the convergence angle is set by the aperture size. **In MuSTEM, the convergence angle is determined by the aperture size, in units of milliradians.** 

<span style="color:orange">Below, in the user input section, input three different convergence angle values in milliradians, with precision to the first decimal place. A reasonable experimental range is between 0.5-30.0 mrad. </span>

[Go to top](#toc)

In [ ]:
#...Activity: MuSTEM series for different aperture sizes...#

#---- User input: Make a list of aperture sizes ----#

aperture = [?, ?, ?] #semiangle in mrad  Ex: [1.0, 3.5, 10.0]

#------------------ End user input -----------------#

#.Setting driver file parameters
driver_name = folder_path + "Drivers/MoS2_mono_driver.txt" #<-- Output driver name
outfnm = folder_path + "Results/2p1_Apertures/MoS2_mono" #<-- Output filename prefix
infnm = folder_path + "Xtl_files/MoS2_mono.xtl" #<-- input xtl filename

kV = "80" #accelerating voltage
numSlice = "4" #number of slices per unit cell
thickness = "3.06300" #Thickness in Angstrom. For series: "Thickness start:Thickness end:Step"
tile = ["26", "15"] #Tile supercell in [X, Y]
numPix = ["512", "512"] #Number of pixels, [X, Y] (pwr of 2)
#aperture = ""  #aperture semiangle in mrad (specified above)
doTilt = [["1","0","0"],["2","0","0"]] #tilt: [["1"=beam , polar, azimuthal], ["2"=specimen, pol., azimuth.]] (mrad)

params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

#.Run the simulation for each aperture size
for n1 in range(len(aperture)):
    params[7] = str(aperture[n1]) #<-- set aperture param. for each value in list. 
    params[0] = outfnm + "_Ap"+params[7] #<-- define custom output filename
    print("\nAperture size: " + params[7])
    make_driver_cbed(driver_name, params, device_type) #<-- make driver file 
    
    run_mustem(mustem_path + " nopause", False) #<-- *Run MuSTEM*
    
print("\nDone.")

#.Read in and plot all cbed files in the produced folder
fig, ax = plt.subplots(1,len(aperture), figsize=(15, 7))
crop_size = 200 #<-- Crop CBED size
cbed=[]

for n1 in range(len(aperture)):
    cbed= load_files(outfnm+"_Ap"+str(aperture[n1]))
    cbed_log=log_cbed(cbed)
    ax[n1].imshow(crop_cbed(cbed_log,crop_size), cmap='CMRmap')
    ax[n1].set_title("Aperture = "+str(aperture[n1]) +" mrad", fontsize=20)


### <span style="color:orange">Checkpoint 5</span>

* How does the size of the CBED disks change with the `aperture` parameter? 

* What does it look like when the convergence angle of the probe becomes bigger than the MoS<sub>2</sub> Bragg angle? (When the size of the probe becomes smaller than the MoS<sub>2</sub> interatomic distance.)

* Can you think of instances where the different convergence angles would be experimentally helpful?

* Which aperture size is associated with the smallest electron probe? What determines the relationship between probe size and CBED disk size? 

[Go to top](#toc)

## <span style="color:orange">2.2. Changing the supercell size </span><a name="section2.2"/>

Another important parameter to consider is the ***`tile size`***. This determines how many times the unit cell is tiled in each direction to form a supercell. If your unit cell is small (say, <30 Angstroms in length), the scattered probe may wrap around and interfere with itself, producing artifacts. In this case, you have to tile your unit cell in the *x* and *y* directions to form a supercell. 

Below are examples of the MoS2 unit cell tiled 12x7, 38x22, and 8x8 times. Note that the MoS<sub>2</sub> unit cell here is not square: it has dimensions of 3.16 A x 5.47 A.

<div>
<img src="Figures/MoS2_Supercell.png" width="700"/>
</div>

<span style="color:orange">In this section we will gain some intuition for the effects of varying the supercell size in the CBED simulation. Run the cell below to produce CBED patterns from monolayer MoS<sub>2</sub> using tile sizes of 12x7, 38x22, and 8x8. Before looking at the results: what do you predict the CBED pattern from the equally-tiled 8x8 supercell will look like?</span> 

[Go to top](#toc)

In [ ]:
#...Activity: MuSTEM series for different supercell sizes...#

#---------------- User input: Make a list of tile sizes ----------------#

tile_size = [["12","7"],["38","22"], ["8","8"]] #tile unit cell in [x,y]

#--------------------------- End user input ----------------------------#

#.Set driver file parameters
driver_name = folder_path + "Drivers/MoS2_mono_driver.txt" #<-- Output driver filename
outfnm = folder_path + "Results/2p2_Supercells/MoS2_mono" #<-- Output filename prefix
infnm = folder_path + "Xtl_files/MoS2_mono.xtl" #<-- Input xtl filename

kV = "80" #accelerating voltage
numSlice = "4" #number of slices per unit cell
thickness = "3.06300" #Thickness in Angstrom. 
tile = ["", ""] #Tile supercell in [X, Y] (specified above)
numPix = ["512", "512"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "3"  #aperture semiangle in mrad
doTilt = [["1","0","0"],["2","0","0"]] #Tilt 

params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

for n1 in range(len(tile_size)):
    params[5] = tile_size[n1]  #<-- Update tile size as list iterates
    params[0] = outfnm + "_Sc"+params[5][0]+"x"+params[5][1] #<-- Customize output filename
    print("\nSupercell size: " + params[5][0]+"x"+params[5][1])
    make_driver_cbed(driver_name, params, device_type) #<-- Make the driver file
    
    run_mustem(mustem_path + " nopause", False) #<-- **Run MuSTEM**
    
print("\nDone.")

#.Load the resulting diffraction patterns
fig, ax = plt.subplots(1,len(tile_size), figsize=(20, 10))
crop_size = 256
for n1 in range(len(tile_size)):
    cbed= load_files(outfnm + "_Sc"+tile_size[n1][0]+"x"+tile_size[n1][1])
    cbed_log=log_cbed(cbed)
    ax[n1].imshow(crop_cbed(cbed_log,crop_size),cmap='CMRmap')
    ax[n1].set_title("Supercell: "+tile_size[n1][0]+"x"+tile_size[n1][1], fontsize=20)

### <span style="color:orange">Checkpoint 6</span>

* For a fixed aperture size, which of the two square supercells has larger and more separated CBED disks? When might you prefer having the 12x7 vs. the 38x22 supercell?

* What happened to the shape of the Bragg disks in the supercell where the *x* and *y* directions were tiled by the same amount? 

### <span style="color:orange">Meaning of supercell size</span>

The **size** of the final supercell affects the size and separation of the CBED disks. A larger supercell has bigger and more separated disks, which improves your diffraction-space resolution. However, for a fixed diffraction grid size, a larger unit cell comes at the expense of less high-angle scattering information (since you have a fixed diffraction space grid, but more widely-spaced CBED disks). For those who have worked with TEMs before, increasing the supercell size is sort of like increasing the camera length on the microscope. 

The **shape** of the supercell determines the shape of the Bragg disks. That's why it's essential to tile your original unit cell such that the final supercell is as close to a square as possible, for circular Bragg disks! In the case of the monolayer MoS2, the rectangular unit cell has a size of 3.16 A x 5.47 A, and so we will have to include more unit cells along the *x* direction to create a roughly square shape. 


Feel free to change the supercell tiling values here when you are done with this section; note that a larger supercell will take longer to run.

[Go to top](#toc)

## <span style="color:orange">2.3. Effects of specimen thickness</span><a name="section2.3"/>

Many of the samples you'll be interested in are not 2D materials. The multislice method can also be used to simulate thicker crystals. 

During the lectures, you might have learned about structure factors and forbidden reflections. In some crystal symmetries, the structure factors from certain reciprocal lattice peaks cancel out such that no reflections occur at those points. For example, in face-centered cubic (FCC) crystals, the peak indices (h, k, l) need to be all odd or all even to have Bragg reflections (i.e. (100) will not have diffraction intensity, but (200) will).

As the crystal thickness increases, however, so too does the contribution to the diffraction pattern from multiple scattering, which is when an electron experiences a scattering incident more than once before exiting the sample. The multiple scattering can result in the appearance of forbidden reflections. We can understand this effect by thinking of the forbidden reflections as vector sums of allowed Bragg reflections. Each term in the vector sum represents one electron scattering incident in the multiple scattering. 

The silicon lattice has some forbidden reflections. In this section, we will explore the effects of thickness and multiple scattering on the silicon [110] CBED pattern, shown below.

<div>
<img src="Figures/Silicon110_2.png" width="400"/>
</div>

In MuSTEM, it's simple to run a series of simulations at different thicknesses. In the driver file, set ***`thickness = A:B:C`*** where ***`A = initial thickness`***, ***`B = final thickness`***, and ***`C = thickness step`***, all in Angstroms. 

<span style="color:orange">The cell below will run a CBED simulation on Silicon (down the [110] zone axis) at ten different thicknesses ranging from 10 to 910 nanometers. When you're done with this section, feel free to run your own thickness series. **Important: if you run this cell with different thickness values, remember to change the output file name of the simulation, or else the** `load_files_series` **method will get confused!** </span>

*Disclaimer: For very thick crystals, it's better to use the `QEP` mode in MuSTEM, not simply the absorptive model. However, that takes much longer to run, and we refer you to the MuSTEM manual to learn more about it.*

[Go to top](#toc)

In [ ]:
# ... Silicon [110] thickness series ... #

#---------------- User input: Make a thickness series ----------------#

thickness = "10:910:100" #<-- Thickness series: "Thickness start:Thickness end:Step" (Angstroms)

#-------------------------- End user input ---------------------------#

#.Define output and input file names
outfnm = folder_path + "Results/2p3_Thickness_Si110/Si" #output filename prefix
driver_name = folder_path + "Drivers/Silicon_110_Thickness.txt"
infnm = folder_path + "Xtl_files/Si_110.xtl" #input xtl filename

#.Set remaining driver parameter files
kV = "300" #accelerating voltage
numSlice = "4" #number of slices per unit cell
tile = ["24", "17"] #Tile supercell in [X, Y]
numPix = ["1024", "1024"] #Number of pixels, [X, Y] (pwr of 2)
#thickness = "" #Thickness (defined above)
aperture = "1.5"  #aperture semiangle in mrad
doTilt = [["1","0","0"],["2","0","0"]] #Tilt 

params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]
make_driver_cbed(driver_name, params, device_type) #<-- make the driver file

#.Run the MuSTEM code
run_mustem(mustem_path + " nopause", True)  #<-- **run MuSTEM**

<span style="color:orange">Run the cell below to load the silicon diffraction patterns you just simulated. You will need to put in the starting thickness, ending thickness, and step size from the previous cell. </span>

Once you have completely finished Section 2.3, feel free to come back and change the ***`thickness`*** parameter! Remember to change the output filename each time you run the simulation again.

[Go to top](#toc)

In [ ]:
#... View silicon thickness series...#

#----------- User input here! ------------------#
thick_start = ? #<-- Starting thickness in Angstrom
thick_step = ? #<-- Step size in Angstrom
thick_end = ? #<-- Ending thickness in Angstrom
crop_size = 200 #<--Cropped size of final diffraction pattern
#--------------- End user input ----------------#

#.Load files
fnm_out=outfnm+"_z="  #<-- search for all files containing the thickness identifier `z=`
cbed = load_files_series(fnm_out)

#. Make list of thicknesses
thick_list = list(range(thick_start, thick_end+thick_step, thick_step))

#.Plot diffraction patterns
ax_len = math.ceil(len(thickness)/5)
fig, ax = plt.subplots(ax_len,5,figsize=(15, 7))
cnt = 0
for n1 in range(ax_len):
    for n2 in range(5):
        ax[n1,n2].imshow(log_cbed(crop_cbed(cbed[cnt][:][:],crop_size)), vmin=-14, vmax=-6, cmap='CMRmap')
        ax[n1, n2].set_title("Thickness: "+str(thick_list[cnt])+" A")
        cnt = cnt+1
plt.tight_layout()

### <span style="color:orange">Checkpoint 7</span>

* What do you notice happening as the thickness increases? 

* Looking at the series, can you make a guess about the crystal thickness of the example Si[110] CBED pattern we showed you above? 

* Can you find any forbidden reflections just by looking at the thickness series? 

### <span style="color:orange">Plotting the intensity of allowed and forbidden reflections in Si [110]</span>

We can differentiate between forbidden and allowed reflections in a thickness series by looking at the intensity of peaks vs. thickness. The intensity of forbidden reflections roughly increases with crystal thickness, because the effects of multiple scattering also increase with specimen thickness. On the other hand, the allowed Bragg reflections will roughly oscillate as a function of thickness. In the next two cells, you will plot the intensities of different CBED disks in the Si[110] diffraction pattern to identify forbidden reflections. 

<span style="color:orange">Run the cell below and identify the coordinates of two CBED disks: one forbidden reflection, and one allowed reflection (make your best guess if you're not sure!). You can use the grid labels at the sides to roughly estimate the coordinates. Input the peak positions in the format *(column, row)* i.e. *(horizontal, vertical)* in the `coords_1` and `coords_2` variables in the cell after this. Round to the nearest integer. </span>

[Go to top](#toc)

In [ ]:
fig = plt.figure(1)
plt.imshow(log_cbed(crop_cbed(cbed[5][:][:],crop_size)), vmin=-14, vmax=-6, cmap='CMRmap')
fig.set_figwidth(8)
fig.set_figheight(8)
plt.title("Find the coordinates of a forbidden and an allowed reflection")

In [ ]:
#... Plot CBED disk intensity ...#

#------- User input here -------#
coords_1 = [?, ?]  #coordinate of peak 1
coords_2 = [?, ?] #coordinate of peak 2
rad = 11  #CBED disk radius
#------- End user input --------#

#. Sum over disk
cbed_sum_1 = []
cbed_sum_2 = []

for n1 in range(len(cbed)):
    tmp=crop_cbed(cbed[n1][:][:],crop_size)
    cbed_sum_1.append(np.sum(tmp[coords_1[1]-rad:coords_1[1]+rad, coords_1[0]-rad:coords_1[0]+rad]))
    cbed_sum_2.append(np.sum(tmp[coords_2[1]-rad:coords_2[1]+rad, coords_2[0]-rad:coords_2[0]+rad]))


#. Plots
fig1, ax = plt.subplots(1,3,figsize=(10, 7))
ax[0].set_title('diffraction pattern')
ax[0].imshow(log_cbed(tmp), vmin=-13, vmax=-6, cmap='inferno')
rect1 = patches.Rectangle((coords_1[0]-rad, coords_1[1]-rad), rad*2, rad*2, linewidth=1, edgecolor='r', facecolor='none')
rect2 = patches.Rectangle((coords_2[0]-rad, coords_2[1]-rad), rad*2, rad*2, linewidth=1, edgecolor='g', facecolor='none')
ax[0].add_patch(rect1)
ax[0].add_patch(rect2)
ax[1].imshow(tmp[coords_1[1]-rad:coords_1[1]+rad, coords_1[0]-rad:coords_1[0]+rad])
ax[1].set_title("Peak 1")
ax[2].imshow(tmp[coords_2[1]-rad:coords_2[1]+rad, coords_2[0]-rad:coords_2[0]+rad])
ax[2].set_title("Peak 2")

fig2, ax = plt.subplots(2,1, figsize=(10,7))
ax[0].set_title("Intensity of Peak 1");
ax[1].set_title("Intensity of Peak 2");
ax[0].plot(np.arange(0, len(cbed), step=1), cbed_sum_1, label="line 1", color='r', marker='o')
ax[1].plot(np.arange(0, len(cbed), step=1), cbed_sum_2, label="line 2", color='g', marker='o')
ax[0].xaxis.set_ticklabels([])
ax[1].xaxis.set_ticklabels(thick_list)  # Set label locations.
ax[0].set_xlim(0,len(cbed)-1)
ax[1].set_xlim(0,len(cbed)-1)
ax[1].set_xlabel('Thickness (A)')
for i in range(2):
    ax[i].set_ylabel('Summed peak intensity')
fig2.set_figwidth(10)
fig2.set_figheight(10)


### <span style="color:orange">2.3. (continued): Plotting the intensity of allowed and forbidden reflections in Si [110]</span>

Do you see how the forbidden reflection increases in intensity at larger thickness, while the allowed reflection oscillates?  The intensity of the forbidden reflection with respect to an allowed reflection can help you estimate your sample thickness. We often use CBED simulations to help us find the thicknesses of our FIB samples.

If you weren't able to find the forbidden reflections, they are circled in the image below.   

<div>
<img src="Figures/Silicon110_ForbiddenReflections_2.png" width="700"/>
</div>

[Go to top](#toc)

## <span style="color:orange">2.4. Artifacts from crystal mistilt</span><a name="section2.4"/>

In reality, we are rarely exactly down zone axis. Mistilts from the exact Bragg condition can lead to artifacts in the recorded diffraction pattern, and CBED simulations can help in visualizing the effects of mistilt in your experimental data. 

Diffraction peaks are formed by the intersection of the Ewald sphere with the crystal's reciprocal lattice points. A slight mistilt from a zone axis leads to a deviation ("excitation error") of the Ewald sphere from these reciprocal lattice points. This deviation can lead to missing peaks, broken inversion symmetry, and other artifacts, depending strongly on the crystal thickness.  

In this section, you will explore and compare the effects of applying mistilt to a thick specimen vs. a thin one. If the effects of mistilt are frustrating to you, keep an eye out for the strain mapping tutorial coming later on this week! :)



### <span style="color:orange">2.4.1 Silicon [110]: Thick specimen</span><a name="section2.4.1"/>

MuSTEM has two tilt options: beam tilt, and specimen tilt. Beam tilt shifts the location of the diffracted beam on the "detector", while specimen tilt rotates the crystal. You have two rotation degrees of freedom for both options: polar tilt, and azimuthal rotation, as defined in spherical coordinates. The values are input in units of milliradians. 

Here you will run a tilt series on 20 nm-thick Si [110]. 

<span style="color:orange">Run the cell below to tilt the Si [110] lattice by 0 to 2.5 degrees about the *x* axis. </span>

[Go to top](#toc)

In [ ]:
#Make driver file for tilt series, Si [110]

#--------------- User input: tilt --------------#

mistilts = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5] #Tilt in degrees 

#------------- End user input ------------------#

#Make a driver file for CBED in MuSTEM 
outfnm = folder_path + "Results/2p4_Tilt/Si110" #output filename prefix
driver_name = folder_path + "Drivers/Si_110_tilting_driver.txt"
infnm = folder_path + "Xtl_files/Si_110.xtl"

kV = "300" #accelerating voltage
numSlice = "5" #number of slices per unit cell
thickness = "200" #Thickness in Angstrom. 
tile = ["24", "17"] #Tile supercell in [X, Y]
numPix = ["1024", "1024"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "1.5"  #aperture semiangle in mrad
doTilt = [["1","0","0"],["2","0","0"]] #Tilt: ["type", "polar", "azimuthal "]. Type "1": beam, "2": specimen (mrad)

params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

As you saw above, the notation we are using here for tilting is: ***`doTilt = [["1", "polar", "azimuth.], ["2", "polar", "azimuth."]]`*** where `"1"` is beam tilt, `"2"` is specimen tilt, and all angles are in milliradians. 

Now run the simulation for each tilt value:

In [ ]:
#.Run MuSTEM tilt series
for n1 in range(len(mistilts)):
    tilt_mrad = "{:0.4f}".format(np.deg2rad(mistilts[n1])*1000) #convert degrees to milliradians (rounded)
    params[8][1][1] = tilt_mrad
    params[0] = outfnm + "_Tilt_"+"{:0.2f}".format(mistilts[n1])
    print("\nMistilt angle (deg) : " + "{:0.2f}".format(mistilts[n1]))
    print("Mistilt angle (mrad) : " + tilt_mrad)
    make_driver_cbed(driver_name, params, device_type)
    
    run_mustem(mustem_path + " nopause", False)
print("\nDone.")

Finally, load the Silicon [110] simulations you just ran. 

[Go to top](#toc)

In [ ]:
#.Load files:
cbed = load_files_series(outfnm)

#.Plot diffraction patterns
dim1 = math.ceil(len(mistilts)/3)
fig, ax = plt.subplots(dim1,3,figsize=(15, 10))
crop_size = 600
cnt = 0
for n1 in range(dim1):
    for n2 in range(3):
        ax[n1, n2].imshow(log_cbed(crop_cbed(cbed[cnt][:][:],crop_size)), vmin = -15, vmax=-9, cmap='CMRmap')
        ax[n1, n2].set_title("Mistilts: "+str(mistilts[cnt])+" deg")
        ax[n1, n2].set_axis_off()
        cnt = cnt+1
plt.tight_layout()

### <span style="color:orange">Checkpoint 8</span>

* What do you notice happening to the diffraction pattern as the crystal tilts? 

* What is causing that circular arc to appear at higher tilt angles?

* How would you expect these effects differ for a thicker crystal? For a thinner one? In which case would you see the effects of mistilt on the CBED pattern more quickly? 

**In thicker crystals, tilting even a little causes asymmetry and missing peaks in the diffraction pattern.** As the crystal is tilted farther and farther from zone axis, one side of the Ewald sphere begins to intersect with many reciprocal lattice points, while the other side intersects with only a few. As a result, a circular arc representing the shape of the Ewald sphere begins to appear at higher tilt angles. 

Feel free to try different a different set of tilts for silicon after this. However, remember to use a different output filename if you run the simulation again!

### <span style="color:orange">2.4.2 Graphene tilt series: Thin specimen</span>   <a name="section2.4.2"/> 

You just saw the effects of tilt on Si [110] of 20 nm thickness. Now we'll look at a thinner material. Unlike in thicker specimens, where the reciprocal lattice is comprised of points localized in three dimensions, as the specimen gets thinner, the reciprocal lattice points broaden into rods extending perpendicular to the 2D plane (i.e. the beam direction). These rods are called "rel-rods". Even if there is some mistilt, the Ewald sphere will still intersect with the rel-rods. Therefore, diffraction patterns from 2D materials tend to be more forgiving to small deviations from zone axis. 

However, in this section you will see that misleading tilt artifacts can still occur in thin specimens. 

<span style="color:orange">Run the cell below to simulate CBED patterns from trilayer (ABC-stacked) graphene tilted 0 to 2 degrees.  </span>

(Note that this series takes a few minutes to run.) 
    
[Go to top](#toc)

In [ ]:
#Graphene tilt series

#--------------- Start user input ----------------#
mistilts = [0.0, 1.0, 2.0] #<-- Tilting angle, in degrees. 
#----------------- End user input ----------------#

#Make a driver file for CBED in MuSTEM 
outfnm = folder_path + "Results/2p4_Tilt/Graphene_ABC" #<-- output filename prefix
driver_name = folder_path + "Drivers/01_Graphene_ABC_driver.txt" #<-- input driver file
infnm = folder_path + "Xtl_files/Graphene_ABC.xtl"

kV = "80" #accelerating voltage
numSlice = "10" #number of slices per unit cell
thickness = "13.5" #Thickness = 1 unit cell.
tile = ["38", "22"] #Tile supercell in [X, Y]
numPix = ["1024", "1024"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "4"  #aperture semiangle in mrad
doTilt = [["1", "0", "0"], ["2", "0", "0"]] #changed below

params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

Now write a driver file and run a simulation for each tilt:

In [ ]:
for n1 in range(len(mistilts)):
    tilt_mrad = "{:0.4f}".format(np.deg2rad(mistilts[n1])*1000) #convert degrees to milliradians (rounded)
    params[8][1][1] = tilt_mrad
    params[0] = outfnm + "_Tilt_"+"{:0.2f}".format(mistilts[n1])
    print("\nMistilt angle (deg) : " + "{:0.2f}".format(mistilts[n1]))
    print("Mistilt angle (mrad) : " + tilt_mrad)
    make_driver_cbed(driver_name, params, device_type)
    run_mustem(mustem_path + " nopause", False)
print("\nDone.")

Run the cell to load the simulated CBED patterns for each tilt. Below each pattern is its difference image, which is the original pattern minus the pattern rotated 180 degrees. 

[Go to top](#toc)

In [ ]:
#Load files:
cbed = load_files_series(outfnm)

#Plot diffraction patterns
fig, ax = plt.subplots(2,len(mistilts),figsize=(15, 10))
crop_size = 200

for n2 in range(len(mistilts)):
    ax[0,n2].imshow(log_cbed(crop_cbed(cbed[n2][:][:],crop_size)), vmin=-15, vmax=-9, cmap='CMRmap')
    ax[0,n2].set_title("Mistilts: "+str(mistilts[n2])+" deg")
    ax[0,n2].set_axis_off()
    
    #.Rotate and subtract the diffraction pattern 
    cbed_r180 = np.roll(np.rot90(cbed[n2][:][:],2),(1,1), axis=(0,1)) 
    cbed_mr180 = np.subtract(cbed[n2][:][:], cbed_r180)  
    
    ax[1,n2].imshow(crop_cbed(cbed_mr180,crop_size), cmap='RdBu_r', vmin=-5.5e-6, vmax=5.5e-6)
    ax[1,n2].set_title("Difference: "+str(mistilts[n2])+" deg")
    ax[1,n2].set_axis_off()
    
plt.tight_layout()

### <span style="color:orange">Checkpoint 9</span>

* What happens to the symmetry of the graphene diffraction pattern as the specimen is tilted? 

* How does this differ from what happened in the Si [110] case? How does the asymmetry differ from the case in the MoS<sub>2</sub> ([Section 1.2](#section1.2))?


Though thin samples are less affected by mistilts than thicker crystals, misleading artifacts due to tilt can still occur. Most notably, **tilt can cause an asymmetry in the intensity of Friedel pairs, even in centrosymmetric crystals.**If you were trying to detect polarity in your CBED pattern, this could be a big problem!

[Go to top](#toc)

#  <span style="color:green">Section 3: Simulating HAADF STEM images</span><a name="section3"/>

Why stop at simulating just one diffraction pattern? If split our sample into a fine grid and generate a diffraction pattern at each point, we can sum over each diffraction pattern using an annular mask to generate atomic-resolution HAADF, ABF, and BF STEM images. 

Here we demonstrate a basic framework for simulating two standard imaging modes -- HAADF (High-angle annular bright field), and ABF (Annular bright field) -- in MuSTEM. Both image types are generated through a virtual annular detector, with the difference being the size and location of the inner and outer rings. 

HAADF images are formed by summing over the extremly high-angle scattering (starting from ~4-5 times the beam semi-convergence angle). The signal is dominated by scattering from the nucleus (Rutherford scattering) and so is very sensitive to atomic number Z: the contrast is roughly proportional to Z<sup>2</sup>. This makes it excellent for imaging heavier elements, but not great for imaging light ones, such as carbon or oxygen. 

ABF images are formed by summing over an annular region within the central, unscattered disk. The contrast of an ABF image is roughly the inverse of a HAADF image, with lighter atoms appearing as brighter.  

In this short section, you will learn how to simulate different STEM imaging modes using the SrTiO<sub>3</sub> structure as a test case. You will already be familiar with most of the parameters from the CBED section!

<div>
<img src="Figures/STO_001.png" width="600"/>
</div>

[Go to top](#toc)

## <span style="color:green">3.1. Making a STEM simulation driver file </span> <a name="section3.1"/>

Much of the driver file for STEM image simulation is similar to the CBED simulations we performed in the previous section. There are some new parameters to modify: 

> 1. ***`interpolation`***: The maximum pixel dimension of the image. The final simulation will be interpolated to this image size.
> 2. ***`tile_interpolation`***: The number of times the unit cell will be tiled in the final image (if it doesn't form a square cell, then the max. number of probe positions will apply to the longer side.)
> 3. ***`detector_radii`***: The radii of the annular detector(s), in milliradians. For example, if your convergence angle is `20` mrad, then a bright-field detector could have inner and outer rings at `0` and `5` mrad, while a HAADF detector could have rings at `80` and `200` mrad. These are the values used below. 
>4. ***`aperture`***: This isn't a new parameter, but its value will be different than in the CBED simulations. Recall that the aperture size defines the real-space resolution (big aperture = better resolution). For atomic-resolution STEM experiments, we tend to use an aperture size of 20-30 mrad. 

When simulating annular dark field STEM images, it's important to keep the size of the diffraction grid (***`num_pix`***) large (at least `1024x1024` pixels). This ensures that the high angle scattering in the diffraction pattern can contribute to the final image formation. The rest of the parameters can remain the same as they were in the CBED simulations. 

<span style="color:green">Run the cell below to make a STEM image driver file for HAADF, ABF, and BF modes.</span>

[Go to top](#toc)

In [ ]:
#...Make a driver file for STEM...#

#.Set input and output filenames and locations
driver_name = folder_path + "Drivers/STO_STEM.txt" #<-- The name of the driver file 
outfnm = folder_path + "Results/3p1_STEM/STO_STEM_2" #<-- The location and filename prefix of the final simulation
infnm = folder_path + "Xtl_files/STO.xtl" #<-- The name of the input .xtl file

#.Change microscope and sample parameters
kV = "300" #accelerating voltage
numSlice = "5" #number of slices per unit cell
thickness = "40" #Thickness in Angstrom. For series: "Thickness start:Thickness end:Step"
tile = ["15", "15"] #Tile supercell in [X, Y]
numPix = ["1024", "1024"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "20"  #aperture semiangle in mrad
defocus = "0" #defocus in A
doTilt = [["1","0","0"], ["2","0","0"]] #Tilt (same as CBED)
interpolation = "512" #size of final STEM image (# probe positions)
tile_interpolation = ["10", "10"] #number of times to tile final supercell in STEM image 
detector_radii = [["0","10"], ["10", "20"], ["80", "220"]] #[inner, outer] radii of annular detectors (mrad)
                  
#.Assemble the parameters into one list
params_stem = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, \
               aperture, defocus, doTilt, interpolation, tile_interpolation, detector_radii]

#.Make the driver file using the chosen parameters above
make_driver_stem(driver_name, params_stem, device_type)

#.Run MuSTEM on the driver file you just made
verbose = True #print entire output of MuSTEM to notebook
run_mustem(mustem_path + " nopause", verbose) #For Windows version

#. Load the images you just created
cbed = load_files_series(outfnm)
    
#. Plot the images:
# The # of virtual detectors (defined in driver file) = # of output files
num_img = len(params_stem[12])
fig, ax = plt.subplots(1,num_img,figsize=(12,7))
for n1 in range(num_img):
    ax[n1].imshow(cbed[n1][:][:], cmap='bone')
    ax[n1].set_title("Detector: "+str(n1+1)+"\nAnnular Mask: "+params_stem[12][n1][0]+\
                    "-"+params_stem[12][n1][1]+" mrad")


### <span style="color:green">Checkpoint 9</span> 

* What kind of contrast did you expect to see in each of the imaging modes? Do the results match your expectations (or experience)?

* What parameter sets the magnification? Field of view?

* How do you think you would you set the ring radii to simulate a BF image?

* How would you improve the spatial resolution?

In the HAADF image (rightmost), you should see the strontium and titanium atoms, with the Sr atoms brighter than the Ti due to their greater atomic number. In the ABF image (middle), you should see contrast from all of the atoms (Sr, Ti, O) with the oxygen atoms showing up as brighter than the other two. 

Feel free to experiment with different STEM parameters. Remember, change the output filename each time you run the simulation. 

[Go to top](#toc)

## <span style="color:green">Quick recap</span> <a name="sectionRecap"/>

We hope you've enjoyed this overview of MuSTEM! You might be tired and want to take a short break before continuing the final part of this tutorial. Here's a quick recap of what we learned so far:

**[Section 1: Using CBED simulations to detect broken inversion symmetry in 2D materials](#section1)**  
* [1.1. Setting up and running a basic CBED simulation](#section1.1)
 * MuSTEM needs two important inputs: a .xtl file with the crystal structure and a driver file with the microscope and sample parameters.
 * The CBED driver file allows you to modify (among many things): sample thickness, sample size, aperture size, sample tilt, diffraction mesh size, and number of slices per unit cell. 
* [1.2. Loading and viewing the simulated CBED patterns](#section1.2)
 * We can view the diffraction patterns in linear or log contrast. The log contrast is better for viewing both the unscattered and high-order scattering CBED disks.
 * The diffraction patterns are binary 32-bit real files with big-endian byte order. The dimensions of the files are written in the filename. 
* [1.3. Detecting broken inversion symmetry](#section1.3)
 * We can use diffraction simulation to detect broken inversion symmetry in a material. 

**[Section 2: Varying the simulation parameters to tune your experiment](#section2)**  
* [2.1. Changing the aperture size](#section2.1)
 * The aperture size determines the beam semi-convergence angle in milliradians. The convergence angle sets the size of the diffraction disk. 
* [2.2. Changing the supercell size](#section2.2)
 * The tiling size sets the number of times the unit cell is repeated in the *x* and *y* directions to make a supercell. A supercell of at least 30x30 Angstroms is needed to prevent artifacts from wrapping errors. 
 * If your supercell is bigger than 30x30 A, then you don't *have* to tile it. Ex: if you have a large nonperiodic structure. 
 * A bigger supercell results in larger and more separated CBED disks. (Kind of like increasing camera length.)
 * You must set the tile sizes such that the supercell is square, for circular CBED disks. 
* [2.3. Changing the crystal thickness](#section2.3) 
 * We can run a thickness series by setting `thickness = start: end: step` in Angstroms. 
 * The intensities of the allowed Bragg reflections vs. the forbidden reflections can be used to determine the crystal thickness. 
* [2.4. Changing the crystal tilt](#section2.4) 
 * We can specify the sample tilt precisely in MuSTEM. This allows us to simulate the effects of mistilt on thick and thin samples. In both cases, small mistilts can lead to confusing diffraction-space artifacts. 
 
**[Section 3: Simulating atomic-resolution STEM images](#section3)**  
* [3.1. How to simulate STEM images](#section3.1) 
 * We learned how to simulate atomic-resolution STEM images in different detector modes. 
 
If you have time left, you can move onto Section 4, where you will learn how to set up a diffraction simulation from the very beginning: writing a crystal structure file.
 
 [Go to top](#toc)

# <span style="color:red">Section 4: Building a crystal structure from scratch</span> <a name="section4"/> 

It's hard to get practice at simulating diffraction patterns unless you start from scratch. If you have time during this tutorial, you might find it fun to build your own crystal structure and diffraction simulation from the very beginning! In this section, you will make the .xtl file by inputting information about the atomic coordinates and elements. Then you will run CBED and STEM imaging simulations, deciding the simulation parameters for yourself. 

In the folder ***CrystalStructures/Section4/***, we have provided some ***.vesta*** files showing the crystal structures of different materials. You can use these as reference for your simulation, or feel free to entirely invent a crystal to simulate during this section!

[Go to top](#toc)

##  <span style="color:red">4.1. Making a .xtl file: rectangular unit cell</span> <a name="section4.1"/> 

We will start with the simpler case of a crystal that already has a rectangular unit cell.

If you chose to use a provided crystal structure in the ***CrystalStructures/Section4/*** folder for this section, and have never used Vesta before, first open the short Vesta tutorial we've included under ***Manuals/UsingVesta.pdf***. 

1) Double-click on a ***.cif*** or ***.vesta*** file to open the material in Vesta. 

You should see the crystal structure appear. We've filled out the ```gold``` lattice as an example here. 

2) What are the minimum number of atoms needed to define the unit cell? For example, in the face-centered cubic gold lattice below, the atoms labeled ```1-4``` can be tiled to repeat the rest of the structure. Every other atom in the lattice is formed by a periodic unit cell reptition of those first four. 

<div>
<img src="Figures/GoldLatticeTable_2.png" width="700"/>
</div>

3) Double-click on those unique atoms to get their ```(x, y, z)``` coordinates (not fractional coordinates). 

4) Then, enter those coordinates into the cell below where directed (***```coords_el1```***). If there is more than one type of atom in your chosen unit cell, start a new list of atomic positions. 

5) Also fill in the following variables: element symbol (***```elements```***), atomic number (***```atom_nums```***), unit cell (***```unit_cell```***), and root mean squared atomic displacement (***```displacement```***). The RMS displacement information can be looked up online if you'd like to be very accurate, or for a less accurate simulation you can put something like `0.008` A<sup>2</sup> and not be terribly far off. 

6) If you have more than one element in your chosen material, enter the information for each element separated by commas. 

<span style="color:red">The information for gold is already filled out as an example. As a start, we suggest you try out another metal, like aluminum (FCC structure) or iron (BCC structure), both included in the CrystalStructures folder! </span>
    
* <span style="color:red">Run the cell to see the generated **.xtl** file. </span>

* <span style="color:red">A **.vasp** file will also be generated, under **CrystalStructures/Section4/UserDef/**. Double-click this **.vasp** file to open it in Vesta. Make sure that the structure you generated is correct. If not, edit the information in the cell as needed. It may take a few tries!</span>

[Go to top](#toc)

In [ ]:
#------------- User input here: ------------#

fnm_prefix = "01_Au" #<-- The filename prefix for the simulation

elements = ["Au"] #<-- Symbols for the elements in the crystal   ["Element 1", "El. 2", ...]
atom_nums = ["79"] #<-- Atomic numbers for each element in the list   ["El. 1", "El. 2", ...]
unit_cell = [4.1729, 4.1729, 4.1729] #<-- Unit cell dimensions (a,b,c) in Angstroms (A)
displacement = ["0.008"] #<-- RMS displacements due to temperature in A^2   ["El. 1", "El. 2", ...]

coords_el1 = [[0, 0, 0],
              [2.08564, 2.08564, 0],
              [2.08564, 0, 2.08564],
              [0, 2.08564, 2.08564]]  #<-- Atomic positions [x, y, z] in Angstroms for element 

coords_el2 = []  #<-- Atomic positions [x, y, z] in Angstroms for element 2

#coords_el3, etc.         #<-- Add more elements if you want 
    
coordinates = [coords_el1] #<-- Combined list of all atomic coordinates [coords_el1, coords_el2, ...]

#------------- End user input -------------#

#Make .xtl and .vasp files
fnm_xtl = folder_path + "Xtl_files/Section4/"+fnm_prefix+".xtl"
fnm_vasp = folder_path + "CrystalStructures/Section4/UserDef/"+fnm_prefix+".vasp"

xtl_params = [elements, atom_nums, unit_cell, displacement, coordinates]
make_xtl(xtl_params, fnm_xtl)
make_vasp(xtl_params, fnm_vasp)

#Open .xtl file
print("XTL FILE:\n")
xtl_file = open(fnm_xtl)
print(xtl_file.read())
xtl_file.close()

#Open .vasp file
print("VASP FILE:\n")
xtl_file = open(fnm_vasp)
print(xtl_file.read())
xtl_file.close()

###  <span style="color:red">4.1. (continued): Visualizing the generated crystal structure</span> 

if you used one of the example .cif files provided, does the .vasp file you wrote generate a crystal structure that looks like the original? What happens when you try to tile the unit cell -- does that work? Some examples of incorrect crystal structures are below:

<div>
<img src="Figures/GoldUnitCell_Comparisons5.png" width="800"/>
</div>

When the crystal structure looks correct, you are ready to move onto the next step!

## <span style="color:red">4.2. Simulate the CBED pattern</span> <a name="section4.2"/>

Now that the .xtl file has been set up, we are ready to write a driver file for a CBED simulation. Most of the parameters here have been left blank for you to fill out, using what you learned in the previous sections. 

<span style="color:red">Try running single simulations (not series), until you are satisfied with the result. Remember to either change the output filename or delete the old simulation file, to not confuse the `load_files` method.</span>

[Go to top](#toc)

In [ ]:
#... Write your own driver file ...#

#--------------- User input here ------------------#
fnm_prefix = "02_Au" #<-- The filename prefix for the simulation
kV = "?" #accelerating voltage
numSlice = "?" #number of slices per unit cell
thickness = "?" #Thickness in Angstrom. 
tile = ["?", "?"] #Tile supercell in [X, Y] 
numPix = ["?", "?"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "?"  #aperture semiangle in mrad 
doTilt = [["1","?","?"], ["2","?","?"]] #Tilt: [[Type, Polar, Azimuth.], [...]] where type = 1 (beam), 2(specimen). In degrees

#----------------- End user input ------------------#

#.Make a driver file for CBED in MuSTEM 
driver_name = folder_path +"Drivers/Section4/"+fnm_prefix+".txt"
outfnm = folder_path +"Results/Section4/"+fnm_prefix #output filename prefix
infnm = fnm_xtl #input xtl filename (defined in prev. cell)

params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

#.Run the simulation 
make_driver_cbed(driver_name, params, device_type) #make the driver file
run_mustem(mustem_path + " nopause", True)  #<-- **Comment this line to avoid running simulation**

### <span style="color:red">4.2. (continued)</span>
Load the diffraction pattern -- you may have to adjust the minimum and maximum intensity values to see the peaks properly. If it doesn't look right, try adjusting the parameters until you are satisfied.

In [ ]:
#... Load the generated diffraction patterns...#

#----------------- Start user input -----------------#
crop_size = ?  #<-- Cropped size of final pattern
int_min = ?    #<-- Minimum intensity value (for LOG)
int_max = ?     #<-- Maximum intensity value (for LOG)
#----------------- End user input -------------------#

#Load files
cbed=load_files(outfnm)
plt.imshow(log_cbed(crop_cbed(cbed, crop_size)), vmin=int_min, vmax=int_max, cmap='CMRmap')
plt.title("Diffraction pattern (LOG)")
plt.colorbar()

## <span style="color:red">4.3. Simulate STEM imaging modes</span><a name="section4.3"/>

We can also directly visualize the lattice of your structure by running some STEM image simulations. In the cell below, fill in the STEM driver file parameters, using what you've learned in Section 3, and run the cell to start a STEM simulation.

[Go to top](#toc)

In [ ]:
#... Define your own STEM image driver file ...#

#----- User input for driver file here: -----#
fnm_prefix = "02_Au" #<-- The filename prefix for the simulation
kV = "?" #accelerating voltage
numSlice = "?" #number of slices per unit cell
thickness = "?" #Thickness in Angstrom. For series: "Thickness start:Thickness end:Step"
tile = ["?", "?"] #Tile supercell in [X, Y]
numPix = ["?", "?"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "?"  #aperture semiangle in mrad
doTilt = [["1","?","?"], ["2","?","?"]] #Tilt: [[Type, Polar, Azimuth.], [...]] where type = 1 (beam), 2(specimen). In degrees
defocus = "?" #defocus in A
interpolation = "?" #size of final STEM image
tile_interpolation = ["?", "?"] #number of times to tile final supercell in STEM image 
detector_radii = [["?","?"], ["?", "?"], ["?", "?"]] #[inner, outer] radii of annular detectors (mrad)
verbose = True #<-- If true, print all command line output to notebook

#-------------- End user input --------------#

#Set input and output filenames and locations
driver_name = folder_path +"Drivers/Section4/"+fnm_prefix+"_STEM.txt" 
outfnm = folder_path +"Results/Section4/"+fnm_prefix+"_STEM"
infnm = fnm_xtl #input xtl filename (defined in prev. cell)

#Make driver file from parameters above
params_stem = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, \
               aperture, defocus, doTilt, interpolation, tile_interpolation, detector_radii]

make_driver_stem(driver_name, params_stem, device_type)

#Run MuSTEM on the driver file you just made
run_mustem(mustem_path + " nopause", verbose) #<-- Comment out this line to not run the simulation

#. Load the images you just created
num_img = len(params_stem[12])
cbed = []
for n1 in range(num_img):
    fnm_out = outfnm+"_DiffPlane00"+str(n1+1)
    cbed.append(load_files(fnm_out))
    
#. Plot the images:
fig, ax = plt.subplots(1,num_img,figsize=(12,7))
for n1 in range(num_img):
    ax[n1].imshow(cbed[n1][:][:])
    ax[n1].set_title("Detector: "+str(n1+1)+"\nAnnular Mask: "+params_stem[12][n1][0]+\
                    "-"+params_stem[12][n1][1]+" mrad")
plt.tight_layout()

Feel free to run through sections 4.1-4.3 with different combinations of simulation parameters, and different input crystal structures. For example, RuO2 or SrTiO3 (both structures included in CrystalStructures folder). If you invented some structures, we'd be curious to see them! 

## <span style="color:red"> 4.4. Hexagonal unit cell + MuSTEM from the command line </span><a name="section4.4"/>

Once you are more comfortable with building a simulation for a crystal with a naturally rectangular unit cell, let's up the difficulty level and try the slightly harder case of a hexagonal structure. We will practice with zinc, which is available in the CrystalStructures folder. 

<div>
<img src="Figures/Zinc_Structure.png" width="300"/>
</div>

<span style="color:red">First, let's make the .xtl file again. What are the atomic positions and unit cell size you would need to define to tile the zinc lattice with a rectangular cell? We are following the same method as earlier. Remember to check how the crystal structure looks in Vesta!</span>

[Go to top](#toc)

In [ ]:
#------------- User input here: ------------#

fnm_prefix = "02_Zn" #<-- The filename prefix for the simulation

elements = ["?", "?"] #<-- Symbols for the elements in the crystal   ["Element 1", "El. 2", ...]
atom_nums = ["?", "?"]#<-- Atomic numbers for each element in the list   ["El. 1", "El. 2", ...]
unit_cell = [?, ?, ?] #<-- Unit cell dimensions (a,b,c) in Angstroms (A)
displacement = ["?", "?"] #<-- RMS displacements due to temperature in A^2   ["El. 1", "El. 2", ...]

coords_el1 = [[?, ?, ?],
              [?, ?, ?], 
              ...]  #<-- Atomic positions [x, y, z] in Angstroms for element 1

#coords_el2 = [[?, ?, ?],
#              [?, ?, ?], 
#              ...]  #<-- Atomic positions [x, y, z] in Angstroms for element 2
    
coordinates = [coords_el1] #<-- Combined list of all atomic coordinates [coords_el1, coords_el2, ...]

#------------- End user input -------------#

#Make .xtl and .vasp files
fnm_xtl = folder_path +"Xtl_files/Section4/"+fnm_prefix+".xtl"
fnm_vasp = folder_path +"CrystalStructures/Section4/UserDef/"+fnm_prefix+".vasp"

xtl_params = [elements, atom_nums, unit_cell, displacement, coordinates]
make_xtl(xtl_params, fnm_xtl)
make_vasp(xtl_params, fnm_vasp)

#Open .xtl file
print("XTL FILE:\n")
xtl_file = open(fnm_xtl)
print(xtl_file.read())
xtl_file.close()

Hint: look at the MoS2 crystal structure if you're not sure! When you're satisfied with the crystal structure, define the driver file and run the CBED simulation:

In [ ]:
#... Write your own driver file ...#

#--------------- User input here ------------------#

fnm_prefix = "02_Zn" #<-- The filename prefix for the simulation
kV = "?" #accelerating voltage
numSlice = "?" #number of slices per unit cell
thickness = "?" #Thickness in Angstrom. 
tile = ["?", "?"] #Tile supercell in [X, Y] 
numPix = ["?", "?"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "?"  #aperture semiangle in mrad 
doTilt = [["1","?","?"], ["2","?","?"]] #Tilt: [Type, Polar, Azimuth.] where type = 1 (beam), 2(specimen). mrad

#----------------- End user input ------------------#

#.Make a driver file for CBED in MuSTEM 
driver_name = folder_path +"Drivers/Section4/"+fnm_prefix+".txt"
outfnm = folder_path +"Results/Section4/"+fnm_prefix #output filename prefix
infnm = fnm_xtl #input xtl filename (defined in prev. cell)

params = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, aperture, doTilt]

#.Run the simulation 
make_driver_cbed(driver_name, params, device_type) #make the driver file
run_mustem(mustem_path + " nopause", True)  #<-- **Comment this line to avoid running simulation**

And load the simulated file

In [ ]:
#... Load the generated diffraction patterns...#

#----------------- Start user input -----------------#
crop_size = ?  #<-- Cropped size of final pattern
int_min = ?    #<-- Minimum intensity value (for LOG)
int_max = ?     #<-- Maximum intensity value (for LOG)
#----------------- End user input -------------------#

#Load files
cbed=load_files(outfnm)
plt.imshow(log_cbed(crop_cbed(cbed, crop_size)), vmin=int_min, vmax=int_max, cmap='CMRmap')
plt.title("Diffraction pattern (LOG)")
plt.colorbar()

You can also run the STEM image simulation on the hexagonal structure, if you want:

In [ ]:
#... Define your own STEM image driver file ...#

#----- User input for driver file here: -----#

fnm_prefix = "02_Zn" #<-- The filename prefix for the simulation
kV = "?" #accelerating voltage
numSlice = "?" #number of slices per unit cell
thickness = "?" #Thickness in Angstrom. For series: "Thickness start:Thickness end:Step"
tile = ["?", "?"] #Tile supercell in [X, Y]
numPix = ["?", "?"] #Number of pixels, [X, Y] (pwr of 2)
aperture = "?"  #aperture semiangle in mrad
doTilt = [["1","?","?"], ["2","?","?"]] #Tilt: [Type, Polar, Azimuth.] where type = 1 (beam), 2(specimen). mrad
defocus = "?" #defocus in A
interpolation = "?" #size of final STEM image
tile_interpolation = ["?", "?"] #number of times to tile final supercell in STEM image 
detector_radii = [["?","?"], ["?", "?"], ["?", "?"]] #[inner, outer] radii of annular detectors (mrad)
verbose = True #<-- If true, print all command line output to notebook

#-------------- End user input --------------#

#Set input and output filenames and locations
driver_name = folder_path +"Drivers/Section4/"+fnm_prefix+"_STEM.txt" 
outfnm = folder_path +"Results/Section4/"+fnm_prefix+"_STEM"
infnm = fnm_xtl #input xtl filename (defined in prev. cell)

#Make driver file from parameters above
params_stem = [outfnm, infnm, kV, numSlice, thickness, tile, numPix, \
               aperture, defocus, doTilt, interpolation, tile_interpolation, detector_radii]

make_driver_stem(driver_name, params_stem, device_type)

#Run MuSTEM on the driver file you just made
run_mustem(mustem_path + " nopause", verbose) #<-- Comment out this line to not run the simulation

#. Load the images you just created
num_img = len(params_stem[12])
cbed = []
for n1 in range(num_img):
    fnm_out = outfnm+"_DiffPlane00"+str(n1+1)
    cbed.append(load_files(fnm_out))
    
#. Plot the images:
fig, ax = plt.subplots(1,num_img,figsize=(12,7))
for n1 in range(num_img):
    ax[n1].imshow(cbed[n1][:][:])
    ax[n1].set_title("Detector: "+str(n1+1)+"\nAnnular Mask: "+params_stem[12][n1][0]+\
                    "-"+params_stem[12][n1][1]+" mrad")
plt.tight_layout()

## <span style="color:red">4.5: Moving to the command line</span><a name="section4.5"/>

Previously, we were using pre-recorded driver files. If you have the time and inclincation, you can also construct the driver file real time from a terminal window. This will give you access to many more options than we provided here. 

First, open the `user_input.txt` file in the main folder. Delete everything that's in it, and replace it with ` interactive`. Save it and close the file. 

Next, open a File Explorer window. Navigate to inside the Tutorial4 folder. As illustrated in the picture below, right click on the name of the folder and select "copy address".

Then, open a new command prompt window. Type `cd ` and then paste the path you just copied. Hit enter and you should now be inside the Tutorials folder. 

<div>
<img src="Figures/HowToMuSTEM_2.png" width="700"/>
</div>

To run MuSTEM, copy the MuSTEM path (***`mustem_path`***) defined at the very beginning of the tutorial, paste it into the command line, and then hit enter. If you get an error, double-check your `user_input.txt` file to make sure the driver file path is correct. 

The program will guide you on how to proceed. When you are done, move to the next cell to load the files.

For the output filename prefix, make sure the name starts with `Results/Section4/` for organizational purposes.

[Go to top](#toc)

In [ ]:
#The mustem path for reference
print(mustem_path)

When the simulation is finished running, you can load it here. Previously, we have been hiding the file loading details in the ***`load_files`*** method. Now, we will show you how to read in a simulation file, so that you can do this on your own in the future. 

In the File Explorer, find the output file you just created. It will have a `.bin` extension. Select and copy the filename, then paste it in place of the `?` mark in the ***`out_fnm`*** variable below. If the file was a CBED simulation, uncomment the section that says `If CBED simulation`. If it was a STEM image simulation, uncomment the `If STEM image simulation` section. Run the cell to view the file!

In [ ]:
#... Load the generated diffraction patterns...#

#----------------- Start user input -----------------#
crop_size = ?  #<-- Cropped size of final pattern
int_min = ?    #<-- Minimum intensity value (for LOG)
int_max = ?     #<-- Maximum intensity value (for LOG)
outfnm = folder_path + \
"Results/Section4/?.bin" #<-- Fill in the filename you used here
#----------------- End user input -------------------#

#Load file
with open(outfnm, 'rb') as file:
    dimX = int((outfnm.split("_"))[-1].split("x")[0])
    dimY = int((outfnm.split(str(dimX)+"x"))[1].split(".bin")[0])
    cbed = np.fromfile(file, dtype='>f4').reshape((dimY, dimX))

'''    
#If CBED simulation:
plt.imshow(log_cbed(crop_cbed(cbed, crop_size)), vmin=int_min, vmax=int_max, cmap='CMRmap')
plt.title("Diffraction pattern (LOG)")
plt.colorbar()
'''

''' 
#If STEM image simulation:
plt.imshow(cbed)
plt.colorbar()
'''

# The end! 

This is the end of the tutorial. We hope you enjoyed it. If you have any questions, don't hesitate to ask us!

[Go to top](#toc)